In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
spark = SparkSession.builder.getOrCreate()
irisDF = spark.read\
                .option('inferSchema','true')\
                .option('header','true')\
                .option('delimiter',',').csv('datasets/iris-dataset.txt')
vec = VectorAssembler(inputCols=irisDF.columns[0:4],outputCol='features')
irisDF = vec.transform(irisDF)
indexer = StringIndexer(inputCol='class',outputCol='label')
indexModel = indexer.fit(irisDF)
irisDF = indexModel.transform(irisDF)
#irisDF.show()
irisDF = irisDF.select('features','label')

trainDF, testDF = irisDF.randomSplit([0.75,0.25],seed=1234)

classifier = RandomForestClassifier()
model = classifier.fit(trainDF)
model.save('savedModel')

resultDF = model.transform(testDF)
resultDF.show()

eva = MulticlassClassificationEvaluator(metricName='accuracy') 
accuracy = eva.evaluate(resultDF)
print("Model Accuracy : ",accuracy) #Accurracy 99% olursa her zaman sevinmeli miyiz ? 
#Eger veri düzgün dağılmamış ise precision / recall / f1 gibi değerlere bakılmalı


Py4JJavaError: An error occurred while calling o914.save.
: java.io.IOException: Path savedModel already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:64)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:564)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at

In [24]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier()
model = classifier.fit(irisDF) #Makine Öğrenmesini yapıldığı satır. Buna Model Training adı verilir.
print("Makine öğrendi")
print(model.featureImportances)


Makine öğrendi
(4,[0,1,2,3],[0.0886288357769059,0.01576229572866298,0.3682771444834464,0.5273317240109847])
